## Transfer Learning Model

In [1]:
import torch 
print(torch.backends.mps.is_available())

True


### Necessary Libraries

In [ ]:
import torch.nn as nn
import torch.optim as optim 
from torchvision import datasets, transforms, models 
from torch.utils.data import DataLoader, random_split
import os
import time
from torchvision.models import resnet18, ResNet18_Weights

if torch.backends.mps.is_available(): 
    device = torch.device("mps")

print("Using device: ", device)

Using device:  mps


### Loading dataset and configuration for tensors

### Create validation and testing set in new dataset !!

In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset_path = 'CNRPark-EXT'
train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
dataset_size = len(train_dataset)
print(f"Total images in dataset: {dataset_size}")

Total images in dataset: 22922


In [4]:
val_dataset_path = "Validation_Set"
val_dataset = datasets.ImageFolder(root=val_dataset_path, transform=transform)

test_dataset_path = "Testing_Set"
test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=transform)

In [5]:
print(f"Train set: {len(train_dataset)}")
print(f"Val set:   {len(val_dataset)}")
print(f"Test set:  {len(test_dataset)}")    

Train set: 22922
Val set:   6171
Test set:  6413


### Loading pretrained model

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Cross Entropy loss for binary classification problems 

Optimizer chosen Adam 

### Training model for 10 epochs 

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
train_losses = []
val_accuracies = []
total_time = 0

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    
    model.eval()
    correct = 0
    total = 0 
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs,1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
   
    validation_acc = 100 * correct/total 
    val_accuracies.append(validation_acc)
    
    epoch_time = time.time() - start_time
    total_time += epoch_time
    
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Val Acc: {validation_acc:.2f}% - Time: {epoch_time:.2f}s")
    print(f"Total time of training: {total_time:.2f}s")
    

Epoch 1/20 - Loss: 0.0473 - Val Acc: 99.77% - Time: 100.43s
Total time of training: 100.43s
Epoch 2/20 - Loss: 0.0180 - Val Acc: 99.27% - Time: 98.57s
Total time of training: 199.00s
Epoch 3/20 - Loss: 0.0105 - Val Acc: 99.21% - Time: 98.70s
Total time of training: 297.70s
Epoch 4/20 - Loss: 0.0131 - Val Acc: 99.30% - Time: 98.82s
Total time of training: 396.52s
Epoch 5/20 - Loss: 0.0082 - Val Acc: 99.34% - Time: 99.52s
Total time of training: 496.03s
Epoch 6/20 - Loss: 0.0028 - Val Acc: 99.85% - Time: 98.69s
Total time of training: 594.73s
Epoch 7/20 - Loss: 0.0111 - Val Acc: 99.17% - Time: 98.72s
Total time of training: 693.44s
Epoch 8/20 - Loss: 0.0064 - Val Acc: 98.67% - Time: 98.49s
Total time of training: 791.94s
Epoch 9/20 - Loss: 0.0037 - Val Acc: 98.82% - Time: 98.40s
Total time of training: 890.34s
Epoch 10/20 - Loss: 0.0019 - Val Acc: 99.34% - Time: 99.01s
Total time of training: 989.35s
Epoch 11/20 - Loss: 0.0064 - Val Acc: 99.00% - Time: 99.20s
Total time of training: 1088

### Saving model

In [8]:
torch.save(model.state_dict(), "model_version6.pth")
print("Model successfully saved as model_version6.pth")

Model successfully saved as model_version6.pth
